# Flood Mapping

- Mapping of flooded areas by comparing images taken before and after a flood event.
- Since most flood events are caused by heavy rainfall, optical imagery has limited applications due to cloud cover.
- SAR imagery is not affected by weather and provides a robust and reliable way to detect flooded regions.

In [1]:
%load_ext autoreload
%autoreload 2
%pdb on
%config InlineBackend.figure_format ='retina'
from IPython.core.debugger import set_trace
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

Automatic pdb calling has been turned ON


In [2]:
import ee
import geemap
ee.Initialize()
m = geemap.Map()
m

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

## Data

We are going to use **Sentinel-1 SAR GRD** (C-band synthetic Aperture Radar Ground Range Detected).

### Sentinel-1 

- Consists of 2 satellites: Sentinel-1A and Sentinel-1B.
- Instrument: C-band synthetic aperture radar (SAR) sensor (day and night).
- Spatial resolution: 10m.
- Temporal resolution: 6-12 days.

### Polarisations and Operational Modes

- S1 can be configured to receive specific polarisations. It can transmit a signal either horizontally (H) or vertically (V), and then receives both H and V.
- Images are captured in different polarizations and available as separate bands: HH, VV, HV, VH.
- 3 different operational modes:
    - **Interferometric Wide Swath (IW)**.
    - Extra Wide Swath (EW).
    - Strip-Map (SP).

### Level-1 Products
- SLC (single look complex)
- GRD (Ground range detected)

## Summary

- Most of the world is captured in IW mode.
- Most areas of the world are images with VV and VH polarisations.

<div style="text-align:center;">
    <img style="width:80%" src="static/imgs/S1_coverage.png" />
</div>

## Problem

We will investigate the 2016 Seine-et-Marne flooding (France): 

<div style="text-align:center;">
    <img style="width:50%" src="static/imgs/paris_flooding.jpeg" />
</div>

- Date: May-June 2016.
- Location: Seine-et-Marne, France.
- Cause: Heavy rains.

This script was originally written by Simon Ilyushchenko (GEE team) and adapted by Simon Gascoin (CNRS/CESBIO) and Michel Le Page (IRD/CESBIO):

In [3]:
def to_natural(img):
    """Convert from dB."""
    return ee.Image(10.0).pow(img.select(0).divide(10.0))

def to_db(img):
    return ee.Image(img).log10().multiply(10.0)

def refined_lee(img):
    """Speckle filter (By Guido Lemoine)."""
    
    # img must be in natural units, i.e. not in dB!
    # Set up 3x3 kernels 
    weights3 = ee.List.repeat(ee.List.repeat(1,3), 3)
    kernel3 = ee.Kernel.fixed(3, 3, weights3, 1, 1, False)
    mean3 = img.reduceNeighborhood(ee.Reducer.mean(), kernel3)
    variance3 = img.reduceNeighborhood(ee.Reducer.variance(), kernel3)

    # Use a sample of the 3x3 windows inside a 7x7 windows to determine gradients and directions
    sample_weights = ee.List(
        [[0,0,0,0,0,0,0], 
         [0,1,0,1,0,1,0],
         [0,0,0,0,0,0,0],
         [0,1,0,1,0,1,0],
         [0,0,0,0,0,0,0],
         [0,1,0,1,0,1,0],
         [0,0,0,0,0,0,0]])

    sample_kernel = ee.Kernel.fixed(7, 7, sample_weights, 3, 3, False)

    # Calculate mean and variance for the sampled windows and store as 9 bands
    sample_mean = mean3.neighborhoodToBands(sample_kernel)
    sample_var = variance3.neighborhoodToBands(sample_kernel)

    # Determine the 4 gradients for the sampled windows
    gradients = sample_mean.select(1).subtract(sample_mean.select(7)).abs()
    gradients = gradients.addBands(sample_mean.select(6).subtract(sample_mean.select(2)).abs())
    gradients = gradients.addBands(sample_mean.select(3).subtract(sample_mean.select(5)).abs())
    gradients = gradients.addBands(sample_mean.select(0).subtract(sample_mean.select(8)).abs())

    # And find the maximum gradient amongst gradient bands
    max_gradient = gradients.reduce(ee.Reducer.max())

    # Create a mask for band pixels that are the maximum gradient
    gradmask = gradients.eq(max_gradient)

    # duplicate gradmask bands: each gradient represents 2 directions
    gradmask = gradmask.addBands(gradmask);

    # Determine the 8 directions
    directions = sample_mean.select(1).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(7))).multiply(1)
    directions = directions.addBands(sample_mean.select(6).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(2))).multiply(2))
    directions = directions.addBands(sample_mean.select(3).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(5))).multiply(3))
    directions = directions.addBands(sample_mean.select(0).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(8))).multiply(4))
  
    # The next 4 are the not() of the previous 4
    directions = directions.addBands(directions.select(0).Not().multiply(5))
    directions = directions.addBands(directions.select(1).Not().multiply(6))
    directions = directions.addBands(directions.select(2).Not().multiply(7))
    directions = directions.addBands(directions.select(3).Not().multiply(8))

    # Mask all values that are not 1-8
    directions = directions.updateMask(gradmask)

    # "collapse" the stack into a singe band image (due to masking, each pixel has just one value (1-8) in it's directional band, and is otherwise masked)
    directions = directions.reduce(ee.Reducer.sum())

    sample_stats = sample_var.divide(sample_mean.multiply(sample_mean));

    # Calculate localNoiseVariance
    sigmaV = sample_stats.toArray().arraySort().arraySlice(0,0,5).arrayReduce(ee.Reducer.mean(), [0])

    # Set up the 7*7 kernels for directional statistics
    rect_weights = ee.List.repeat(ee.List.repeat(0,7),3).cat(ee.List.repeat(ee.List.repeat(1,7),4))

    diag_weights = ee.List([
        [1,0,0,0,0,0,0], 
        [1,1,0,0,0,0,0], 
        [1,1,1,0,0,0,0], 
        [1,1,1,1,0,0,0], 
        [1,1,1,1,1,0,0], 
        [1,1,1,1,1,1,0], 
        [1,1,1,1,1,1,1]])

    rect_kernel = ee.Kernel.fixed(7, 7, rect_weights, 3, 3, False)
    diag_kernel = ee.Kernel.fixed(7, 7, diag_weights, 3, 3, False)

    # Create stacks for mean and variance using the original kernels. Mask with relevant direction.
    dir_mean = img.reduceNeighborhood(ee.Reducer.mean(), rect_kernel).updateMask(directions.eq(1))
    dir_var = img.reduceNeighborhood(ee.Reducer.variance(), rect_kernel).updateMask(directions.eq(1))

    dir_mean = dir_mean.addBands(img.reduceNeighborhood(ee.Reducer.mean(), diag_kernel).updateMask(directions.eq(2)))
    dir_var = dir_var.addBands(img.reduceNeighborhood(ee.Reducer.variance(), diag_kernel).updateMask(directions.eq(2)))

    # and add the bands for rotated kernels
    for i in range(1, 4):
        dir_mean = dir_mean.addBands(img.reduceNeighborhood(ee.Reducer.mean(), rect_kernel.rotate(i)).updateMask(directions.eq(2*i+1)))
        dir_var = dir_var.addBands(img.reduceNeighborhood(ee.Reducer.variance(), rect_kernel.rotate(i)).updateMask(directions.eq(2*i+1)))
        dir_mean = dir_mean.addBands(img.reduceNeighborhood(ee.Reducer.mean(), diag_kernel.rotate(i)).updateMask(directions.eq(2*i+2)))
        dir_var = dir_var.addBands(img.reduceNeighborhood(ee.Reducer.variance(), diag_kernel.rotate(i)).updateMask(directions.eq(2*i+2)))

    # "collapse" the stack into a single band image (due to masking, each pixel has just one value in it's directional band, and is otherwise masked)
    dir_mean = dir_mean.reduce(ee.Reducer.sum())
    dir_var = dir_var.reduce(ee.Reducer.sum())

    # A finally generate the filtered value
    varX = dir_var.subtract(dir_mean.multiply(dir_mean).multiply(sigmaV)).divide(sigmaV.add(1.0))

    b = varX.divide(dir_var);

    result = dir_mean.add(b.multiply(img.subtract(dir_mean)))
    return result.arrayFlatten([['sum']])

In [ ]:
# Set location of interest
pt = ee.Geometry.Point(3.02, 48.82)

# Load Sentinel-1 C-band SAR Ground Range collections
imgs = ee.ImageCollection("COPERNICUS/S1_GRD")\
            .filter(ee.Filter.geometry(pt))\
            .filter(ee.Filter.listContains("transmitterReceiverPolarisation", "VV"))\
            .select("VV")

# Filter by date to get the "before" & "after" images
before = imgs.filterDate("2016-05-01", "2016-05-17").mosaic()
after = imgs.filterDate("2016-05-30", "2016-06-01").mosaic()

# Threshold smoothed radar intensities to identify `flooded` areas
SMOOTHING_RADIUS = 100
DIFF_UPPER_THRESHOLD = -3

diff_smoothed = after.focal_median(SMOOTHING_RADIUS, "circle", "meters")\
    .subtract(before.focal_median(SMOOTHING_RADIUS, "circle", "meters"))

# Remove speckle noise and calculate the difference between "before" and "after"
diff_smoothed_lee = to_db(refined_lee(to_natural(after))).subtract(to_db(refined_lee(to_natural(before))))

diff_thresholded = diff_smoothed.lt(DIFF_UPPER_THRESHOLD)
diff_thresholded_lee = diff_smoothed_lee.lt(DIFF_UPPER_THRESHOLD)

# Visualize
m.centerObject(pt, 13)
m.addLayer(before, {"min": -30, "max": 0}, "Before flood", 0)
m.addLayer(after, {"min": -30, "max": 0}, "After flood", 0)
m.addLayer(after.subtract(before), {"min": -10, "max": 10}, "After - before", 0)
m.addLayer(diff_smoothed, {"min": -10, "max": 10}, "diff smoothed", 0)
m.addLayer(diff_smoothed_lee, {"min": -10, "max": 10}, "diff smoothed refined Lee", 0) 
m.addLayer(diff_thresholded.updateMask(diff_thresholded), {"palette": "0000FF"}, "flooded areas", 0)
m.addLayer(diff_thresholded_lee.updateMask(diff_thresholded_lee), {"palette": "0000FF"}, "flooded areas (Lee)", 1)

In [ ]:
geemap.ee_search()

---